In [1]:
import torch
import torch.nn as nn
import os
import cv2
import numpy as np
import pandas as pd
import csv
import matplotlib.pyplot as plt
import warnings
from tqdm import tqdm
warnings.filterwarnings('ignore')

In [2]:
from torchvision import models
from torchvision import transforms
import albumentations as A
from albumentations.pytorch import ToTensorV2
import random
from sklearn.metrics import confusion_matrix

In [3]:
if(os.path.split(os.getcwd())[1] == "Transfer_Learning"):
    os.chdir("..")
print("Current Working Directory: {}".format(os.path.split(os.getcwd())[1]))

cuda = False

Current Working Directory: progettoVIPM


In [4]:
from utils.loadersAndEnums import datasets
from utils.loadersAndEnums import networks
from utils.loadersAndEnums import ImageDataset

In [5]:
model = models.resnet50()
num_classes = 101
model.fc = nn.Linear(model.fc.in_features, num_classes)
if cuda:
    model.load_state_dict(torch.load("Storage/models/resnet50model.pth")['model_state'])
else:
    model.load_state_dict(torch.load("Storage/models/resnet50model.pth", map_location=torch.device('cpu') )['model_state']) 
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
model.to(device)

cpu


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [6]:
num_classes = 251
model.fc = nn.Linear(model.fc.in_features, num_classes)
model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [7]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

In [12]:
random.seed(42)
train_transform = A.Compose(
    [
        A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.05, rotate_limit=15, p=0.5),
        #A.RandomCrop(height=224, width=224, p=0.5),
        A.RGBShift(r_shift_limit=15, g_shift_limit=15, b_shift_limit=15, p=0.5),
        A.RandomBrightnessContrast(p=0.5),        
        A.RandomToneCurve(scale=0.7, p=0.5),
        A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),#as required by All pre-trained models of torch
        ToTensorV2(),
    ]
)


In [13]:
from torch.utils.data import DataLoader
trainData = datasets.TRAINING_LABELED
testData = datasets.TEST
train = ImageDataset(trainData, 224,cuda=cuda, transform=None) # NO MORE the image is loaded bigger than it should be...but the transform will take a random crop of the right dimension(224x224)
train_dataset, val_dataset = torch.utils.data.random_split(train, [0.8, 0.2])
test = ImageDataset(testData,224,cuda=cuda)

trainDL = DataLoader(train_dataset, batch_size=64, shuffle=False)
valDL = DataLoader(val_dataset, batch_size=128, shuffle=False)
testDL = DataLoader(test, batch_size=128, shuffle=False)

In [14]:
if cuda:
    torch.cuda.empty_cache()
    batch = next(iter(trainDL))
    print(batch)

## Supervised Finetuning

In [16]:
num_epochs = 5

for epoch in tqdm(range(num_epochs)):

    model.train()#Set the module in training mode.
    running_loss = 0.0

    for inputs, labels in tqdm(trainDL):
        print("new batch")
        labels = [int(label) for label in labels]
        inputs = inputs.to(device) 
        labels = torch.tensor(labels, dtype=torch.long).to(device)        
        # Zero the parameter gradients
        optimizer.zero_grad()        
        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)        
        # Backward pass and optimization
        loss.backward()
        optimizer.step()        
        running_loss += loss.item()
    
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(trainDL)}')

  0%|          | 0/5 [00:00<?, ?it/s]

new batch


new batch


new batch


new batch


...

new batch


new batch


new batch


100%|██████████| 5/5 [1:12:02<00:00, 864.40s/it]

Epoch 5/5, Loss: 1.5636277217713614


In [20]:
model.eval()
correct = 0
total = 0
all_preds = []
all_labels = []
with torch.no_grad():
    for inputs, labels in tqdm(testDL):
        labels = [int(label) for label in labels]
        inputs, labels = inputs.to(device), torch.tensor(labels, dtype=torch.long).to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        all_labels.extend(labels.cpu().numpy())
        all_preds.extend(predicted.cpu().numpy())
        

all_preds = np.array(all_preds)
all_labels = np.array(all_labels)
cm = confusion_matrix(all_labels, all_preds)

print(f'Accuracy: {100 * correct / total}%')
np.save('cmResNet50PreTrainedAugmented5Epochs.npy',cm)
torch.save(model, "finetunedResNet50Augmented1.pth")


100%|██████████| 94/94 [19:03<00:00, 12.17s/it]

Accuracy: 38.46923461730866%
